In [ ]:
# example notebook for autodo api usage

# NOTE::
# This notebook is primarily for local/dev use.  The ssl_verify flag
#   is used to allow http connections.   The public version which uses
#   APIHub does not expose this flag.

In [ ]:
import requests
import json

AUTODO_API = "https://temporal.sl.cloud9.ibm.com:8443/autodo"
#AUTODO_API = "https://api.ibm.com/autodo/run/autodo"

email = "YOUR EMAIL HERE"

headers = { "Content-type": "application/json" }

# set to None if not using API Hub
creds = {"X-IBM-Client-Id": "YOUR CLIENT ID HERE",
         "X-IBM-Client-Secret": "YOUR CLIET SECRET HERE" }

creds = None

if creds is not None:
    headers.update(creds)
    
# if running w/ a self-signed ssl certificate, the following will disable verification and hide warnings
ssl_verify = False
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(category=InsecureRequestWarning)


In [ ]:
# System Health Check
req = requests.get(f"{AUTODO_API}/health-check", headers=headers, verify=ssl_verify)
print(json.dumps(req.json(), indent=2))

In [ ]:
# get all submissions
# TBD:: will be per user in the future

req = requests.get(f"{AUTODO_API}/submissions?email={email}", headers=headers, verify=ssl_verify)
all_submissions = req.json()
print(json.dumps(all_submissions, indent=2))

In [ ]:
# just the running submissions
print(json.dumps([ x for x in all_submissions if x["status"] == "running" ], indent=2))

In [ ]:
# make a new submission
params = { "email": email,
           "env": "CartPole-v0",
           "action_type": "discrete" }
req = requests.post(f"{AUTODO_API}/submissions", headers=headers, data=json.dumps(params), verify=ssl_verify)
submission = req.json()
print(json.dumps(submission, indent=2))

In [ ]:
# get submission info
req = requests.get(f"{AUTODO_API}/submissions/{submission['submission_uuid']}", headers=headers, verify=ssl_verify)
info = req.json()
print(json.dumps(info, indent=2))

In [ ]:
# get submission artifacts when complete
req = requests.get(f"{AUTODO_API}/submissions/{submission['submission_uuid']}/results",headers=headers, verify=ssl_verify)
print(json.dumps(req.json(), indent=2))

In [ ]:
# delete submission
req = requests.delete(f"{AUTODO_API}/submissions/{submission['submission_uuid']}", headers=headers, verify=ssl_verify)
print(json.dumps(req.json(), indent=2))

In [ ]:
# make user env submission
params = { "action_type": "discrete", "email": email }
path = "../data/dynamic/environments/inventory_test_env_v1p_plus_demand_generator.py"
multipart_form_data = {
    "env": ("env", open(path, "rb")),
    "params": (None, json.dumps(params))
}
req = requests.post(f"{AUTODO_API}/code_submissions", files=multipart_form_data, headers=creds, verify=ssl_verify)
submission = req.json()
print(json.dumps(submission, indent=2))

In [ ]:
# delete all submissions
req = requests.get(f"{AUTODO_API}/submissions?email={email}", headers=headers, verify=ssl_verify)
all_submissions = req.json()
for sub in all_submissions:
    req = requests.delete(f'{AUTODO_API}/submissions/{sub["submission_uuid"]}', headers=headers, verify=ssl_verify)
    print(json.dumps(req.json(), indent=2))

In [ ]:
# listing available agents
req = requests.get(f"{AUTODO_API}/agents", headers=headers, verify=ssl_verify)
print(json.dumps(req.json(), indent=2))
